In [1]:
import warnings
import numpy as np
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

dask.config.set(scheduler='threads') 
warnings.filterwarnings("ignore", category=FutureWarning)
ProgressBar().register()

In [2]:
df = dd.read_csv('data/train_data.csv')
df = df.set_index('customer_ID').repartition(npartitions=256)
labels = dd.read_csv('data/train_labels.csv').set_index('customer_ID')
df.dtypes.to_csv('dtypes.csv')

[########################################] | 100% Completed |  3min 14.1s
[########################################] | 100% Completed |  1.1s


In [3]:
na_stats = df.isna().sum().compute()
na_stats.to_csv("na_stats.csv")

[########################################] | 100% Completed |  2min  4.0s


In [3]:
value_cols = list(set(df.columns.tolist()) - {'customer_ID', 'S_2', 'D_63', 'D_64'})
grouped_df = df.groupby('customer_ID')[value_cols]
means = grouped_df.mean().rename(columns=lambda x: x+ "_mean")
stddev = grouped_df.std().rename(columns=lambda x: x+ "_std")
minv = grouped_df.min().rename(columns=lambda x: x+ "_min")
maxv = grouped_df.max().rename(columns=lambda x: x+ "_max")
feats = means.join(stddev).join(minv).join(maxv).join(labels)

In [5]:
#import functools
#import pandas as pd
#iqr25_fun = functools.partial(np.nanquantile, q=0.25, axis=0)
#iqr25 = df[value_cols].map_partitions(iqr25_fun, meta=pd.DataFrame)

In [5]:
feats.to_parquet('features-dask.parquet', compression='gzip')

[########################################] | 100% Completed |  6min 29.1s


(None,)

In [ ]:
# Process test data
df = dd.read_csv('data/test_data.csv')
df = df.set_index('customer_ID').repartition(npartitions=256)

In [ ]:
value_cols = list(set(df.columns.tolist()) - {'customer_ID', 'S_2', 'D_63', 'D_64'})
grouped_df = df.groupby('customer_ID')[value_cols]
means = grouped_df.mean().rename(columns=lambda x: x+ "_mean")
stddev = grouped_df.std().rename(columns=lambda x: x+ "_std")
minv = grouped_df.min().rename(columns=lambda x: x+ "_min")
maxv = grouped_df.max().rename(columns=lambda x: x+ "_max")
feats = means.join(stddev).join(minv).join(maxv)

In [ ]:
feats.to_parquet('features-dask-test.parquet', compression='gzip')